In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('default')
from metrics import accuracy

from sklearn import datasets
from sklearn.model_selection import train_test_split

# PCA: Principle Component Analysis 

### A bit of Background: Curse of Dimensionality
Many datasets have thousands or even millions of features per training instance - Using all of them will likely slow down your training.     
Consider that:     
Some of the features may not even add much information and could be discarded or ignored.     
Other features may be so highly correlated that you might be able to merge them into one and lose little information.     
   
Furthermore, most people can readily understand 3 dimensions but begin to struggle with 4 dimensions, let alone thousands.    
Other particularities of high dimensional data:
* In a unit square (1x1) points are unlikely to be extreme along any dimension whereas in a high dimensional space (eg. 10.000 dims) most points lay along the border of their spaces hypercube.
* 